## Atividade Prática VI

Este trabalho tem como objetivo aplicar o algoritmo Random Forest na base de dados Madelon, desenvolvida para o NIPS 2003 Feature Selection Challenge. A base apresenta alta dimensionalidade e variáveis irrelevantes, tornando o problema de classificação desafiador. Foram realizados experimentos em dois cenários  "dados normalizados e não normalizados"   e comparadas três abordagens de predição: votação majoritária (hard voting), ponderada (weighted voting) e suave (soft voting).

Preparação do ambiente e importação das bibliotecas

In [42]:
import os
import zipfile
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Download da base Madelon

In [43]:

zip_path = "/home/emilly/Trabalho Topicos_florest/Madelon-RandomForest-Classification/madelon.zip"
extract_path = "/home/emilly/Trabalho Topicos_florest/Madelon-RandomForest-Classification/dataset"

os.makedirs(extract_path, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Arquivos extraídos com sucesso!")
print("Conteúdo da pasta 'dataset/':")
print(os.listdir(extract_path))


Arquivos extraídos com sucesso!
Conteúdo da pasta 'dataset/':
['madelon_valid.labels', 'Dataset.pdf', 'MADELON']


Analise do dataset

In [44]:

base_path = "/home/emilly/Trabalho Topicos_florest/Madelon-RandomForest-Classification/dataset/MADELON"
train_data_path = os.path.join(base_path, "madelon_train.data")
train_labels_path = os.path.join(base_path, "madelon_train.labels")


X = pd.read_csv(train_data_path, sep=" ", header=None)
y = pd.read_csv(train_labels_path, sep=" ", header=None)

X = X.dropna(axis=1, how='all')

print("Dimensões dos dados:")
print(f"X (atributos): {X.shape}")
print(f"y (rótulos): {y.shape}\n")

print("Primeiras linhas dos atributos:")
display(X.head())

print("Distribuição das classes:")
print(y.value_counts().rename_axis("Classe").reset_index(name="Frequência"))

# Estatísticas básicas
print("\n Estatísticas dos atributos (amostra):")
display(X.describe().T.head(10))


Dimensões dos dados:
X (atributos): (2000, 500)
y (rótulos): (2000, 1)

Primeiras linhas dos atributos:


,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,485,477,537,479,452,471,491,476,475,473,...,477,481,477,485,511,485,481,479,475,496
1,483,458,460,487,587,475,526,479,485,469,...,463,478,487,338,513,486,483,492,510,517
2,487,542,499,468,448,471,442,478,480,477,...,487,481,492,650,506,501,480,489,499,498
3,480,491,510,485,495,472,417,474,502,476,...,491,480,474,572,454,469,475,482,494,461
4,484,502,528,489,466,481,402,478,487,468,...,488,479,452,435,486,508,481,504,495,511


Distribuição das classes:
   Classe  Frequência
0      -1        1000
1       1        1000

 Estatísticas dos atributos (amostra):


,count,mean,std,min,25%,50%,75%,max
0,2000.0,481.7225,6.421769,462.0,477.00,482.0,486.00,503.0
1,2000.0,483.4525,30.186294,381.0,464.00,483.0,503.00,600.0
2,2000.0,510.1660,38.899165,370.0,485.00,510.5,536.00,654.0
3,2000.0,483.3845,9.059895,453.0,477.00,483.0,490.00,519.0
4,2000.0,501.6125,41.389418,371.0,475.00,500.0,528.00,688.0
5,2000.0,479.2590,6.795956,459.0,475.00,479.0,484.00,505.0
6,2000.0,480.1095,40.575925,334.0,452.75,480.0,506.25,611.0
7,2000.0,476.5650,1.384461,471.0,476.00,477.0,477.00,481.0
8,2000.0,486.7935,15.043836,430.0,477.00,487.0,496.25,536.0
9,2000.0,478.7890,7.190920,455.0,474.00,479.0,484.00,503.0


##  Cenários de Classificação: 

Dados não normalizados

In [51]:

# Caminho base
base_path = "/home/emilly/Trabalho Topicos_florest/Madelon-RandomForest-Classification/dataset/MADELON"

# Caminhos dos arquivos
train_data_path = os.path.join(base_path, "madelon_train.data")
train_labels_path = os.path.join(base_path, "madelon_train.labels")
valid_data_path = os.path.join(base_path, "madelon_valid.data")
valid_labels_path = os.path.join(base_path, "madelon_valid.labels")

X_train = pd.read_csv(train_data_path, sep=" ", header=None)
y_train = pd.read_csv(train_labels_path, sep=" ", header=None)

X_valid = pd.read_csv(valid_data_path, sep=" ", header=None)
y_valid = pd.read_csv(valid_labels_path, sep=" ", header=None)

# Remover colunas vazias
X_train = X_train.dropna(axis=1, how="all")
X_valid = X_valid.dropna(axis=1, how="all")

X_total = pd.concat([X_train, X_valid], ignore_index=True)
y_total = pd.concat([y_train, y_valid], ignore_index=True).squeeze()

print(f"Tamanho total após junção: {X_total.shape[0]} amostras e {X_total.shape[1]} features")


X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(
    X_total, y_total, test_size=0.2, random_state=42, stratify=y_total
)

print(f"Tamanho treino: {X_train_final.shape[0]} | Tamanho teste: {X_test_final.shape[0]}")

# Modelo Random Forest
modelo = RandomForestClassifier(
    n_estimators=200, random_state=42, n_jobs=-1
)
modelo.fit(X_train_final, y_train_final)
y_pred = modelo.predict(X_test_final)

importances = modelo.feature_importances_
indices = np.argsort(importances)[::-1]
top_features = indices[:20]

X_train_reduzido = X_train_final.iloc[:, top_features]
X_test_reduzido = X_test_final.iloc[:, top_features]

print("============================================")
print("CENÁRIO 1: SEM NORMALIZAÇÃO (HOLD-OUT 80/20)")
print("============================================")
print(f"Acurácia: {accuracy_score(y_test_final, y_pred):.4f}")
print("\nRelatório de Classificação:")
print(classification_report(y_test_final, y_pred))
print("\nMatriz de Confusão:")
print(confusion_matrix(y_test_final, y_pred))


Tamanho total após junção: 2600 amostras e 500 features
Tamanho treino: 2080 | Tamanho teste: 520
CENÁRIO 1: SEM NORMALIZAÇÃO (HOLD-OUT 80/20)
Acurácia: 0.7058

Relatório de Classificação:
              precision    recall  f1-score   support

          -1       0.68      0.77      0.72       260
           1       0.74      0.64      0.69       260

    accuracy                           0.71       520
   macro avg       0.71      0.71      0.70       520
weighted avg       0.71      0.71      0.70       520


Matriz de Confusão:
[[200  60]
 [ 93 167]]


Dados normalizados

In [52]:

base_path = "/home/emilly/Trabalho Topicos_florest/Madelon-RandomForest-Classification/dataset/MADELON"

train_data_path = os.path.join(base_path, "madelon_train.data")
train_labels_path = os.path.join(base_path, "madelon_train.labels")
valid_data_path = os.path.join(base_path, "madelon_valid.data")
valid_labels_path = os.path.join(base_path, "madelon_valid.labels")

X_train = pd.read_csv(train_data_path, sep=" ", header=None)
y_train = pd.read_csv(train_labels_path, sep=" ", header=None)

X_valid = pd.read_csv(valid_data_path, sep=" ", header=None)
y_valid = pd.read_csv(valid_labels_path, sep=" ", header=None)

X_train = X_train.dropna(axis=1, how="all")
X_valid = X_valid.dropna(axis=1, how="all")

X_total = pd.concat([X_train, X_valid], ignore_index=True)
y_total = pd.concat([y_train, y_valid], ignore_index=True).squeeze()

print(f"Tamanho total após junção: {X_total.shape[0]} amostras e {X_total.shape[1]} features")

# Divisão hold-out (80% treino, 20% teste)
X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(
    X_total, y_total, test_size=0.2, random_state=42, stratify=y_total
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_final)
X_test_scaled = scaler.transform(X_test_final)

modelo = RandomForestClassifier(
    n_estimators=300, random_state=42, n_jobs=-1
)
modelo.fit(X_train_scaled, y_train_final)
y_pred = modelo.predict(X_test_scaled)

print("============================================")
print("CENÁRIO 2: COM NORMALIZAÇÃO (HOLD-OUT 80/20)")
print("============================================")
print(f"Acurácia: {accuracy_score(y_test_final, y_pred):.4f}")
print("\nRelatório de Classificação:")
print(classification_report(y_test_final, y_pred))
print("\nMatriz de Confusão:")
print(confusion_matrix(y_test_final, y_pred))

Tamanho total após junção: 2600 amostras e 500 features
CENÁRIO 2: COM NORMALIZAÇÃO (HOLD-OUT 80/20)
Acurácia: 0.7077

Relatório de Classificação:
              precision    recall  f1-score   support

          -1       0.68      0.79      0.73       260
           1       0.75      0.63      0.68       260

    accuracy                           0.71       520
   macro avg       0.71      0.71      0.71       520
weighted avg       0.71      0.71      0.71       520


Matriz de Confusão:
[[205  55]
 [ 97 163]]


# Aplicação do Algoritmo 

Aplicação do algoritmo Random Forest na classificação dos dados utilizando os 
seguintes métodos para determinar a predição:
* votação majoritária
* votação ponderada
* votação suave (soft voting).


CENÁRIO Não NORMALIZADO

In [33]:

base_path = "/home/emilly/Trabalho Topicos_florest/Madelon-RandomForest-Classification/dataset/MADELON"

train_data_path = os.path.join(base_path, "madelon_train.data")
train_labels_path = os.path.join(base_path, "madelon_train.labels")
valid_data_path = os.path.join(base_path, "madelon_valid.data")
valid_labels_path = os.path.join(base_path, "madelon_valid.labels")

X_train = pd.read_csv(train_data_path, sep=" ", header=None)
y_train = pd.read_csv(train_labels_path, sep=" ", header=None)
X_valid = pd.read_csv(valid_data_path, sep=" ", header=None)
y_valid = pd.read_csv(valid_labels_path, sep=" ", header=None)

# Remove colunas vazias
X_train = X_train.dropna(axis=1, how="all")
X_valid = X_valid.dropna(axis=1, how="all")

# Junta treino + validação
X_total = pd.concat([X_train, X_valid], ignore_index=True)
y_total = pd.concat([y_train, y_valid], ignore_index=True).squeeze()

print(f"Tamanho total após junção: {X_total.shape[0]} amostras e {X_total.shape[1]} features")


X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(
    X_total, y_total, test_size=0.2, random_state=42, stratify=y_total
)

modelo = RandomForestClassifier(n_estimators=300, random_state=42)
modelo.fit(X_train_final, y_train_final)

# Predição direta (hard voting)
pred_hard = modelo.predict(X_test_final)

# Probabilidades preditas (para simular votações suaves/ponderadas)
prob = modelo.predict_proba(X_test_final)

# Votação suave (soft voting)
pred_soft = modelo.classes_[prob.argmax(axis=1)]

# Votação ponderada (weighted voting): aplica pesos simulados nas probabilidades
pesos = np.linspace(0.5, 1.0, prob.shape[1])
pred_weighted = modelo.classes_[np.argmax(prob * pesos, axis=1)]

def avaliar(y_true, y_pred):
    return {
        "Acurácia": accuracy_score(y_true, y_pred),
        "Precisão": precision_score(y_true, y_pred, average='weighted', zero_division=0),
        "Recall": recall_score(y_true, y_pred, average='weighted', zero_division=0),
        "F1-Score": f1_score(y_true, y_pred, average='weighted', zero_division=0),
        "Kappa": cohen_kappa_score(y_true, y_pred),
    }


resultados = {
    "Votação Majoritária": avaliar(y_test_final, pred_hard),
    "Votação Suave": avaliar(y_test_final, pred_soft),
    "Votação Ponderada": avaliar(y_test_final, pred_weighted),
}

df_resultados = pd.DataFrame(resultados).T.round(4)
print("\n============================================")
print("RESULTADOS - CENÁRIO NÃO NORMALIZADO (80/20)")
print("============================================")
print(df_resultados)


Tamanho total após junção: 2600 amostras e 500 features

RESULTADOS - CENÁRIO NÃO NORMALIZADO (80/20)
                     Acurácia  Precisão  Recall  F1-Score   Kappa
Votação Majoritária    0.7077    0.7133  0.7077    0.7058  0.4154
Votação Suave          0.7077    0.7133  0.7077    0.7058  0.4154
Votação Ponderada      0.5365    0.7357  0.5365    0.4124  0.0731


CENÁRIO NORMALIZADO

In [34]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score
)


base_path = "/home/emilly/Trabalho Topicos_florest/Madelon-RandomForest-Classification/dataset/MADELON"

train_data_path = os.path.join(base_path, "madelon_train.data")
train_labels_path = os.path.join(base_path, "madelon_train.labels")
valid_data_path = os.path.join(base_path, "madelon_valid.data")
valid_labels_path = os.path.join(base_path, "madelon_valid.labels")

X_train = pd.read_csv(train_data_path, sep=" ", header=None)
y_train = pd.read_csv(train_labels_path, sep=" ", header=None)
X_valid = pd.read_csv(valid_data_path, sep=" ", header=None)
y_valid = pd.read_csv(valid_labels_path, sep=" ", header=None)

# Remove colunas vazias
X_train = X_train.dropna(axis=1, how="all")
X_valid = X_valid.dropna(axis=1, how="all")

# Junta treino + validação
X_total = pd.concat([X_train, X_valid], ignore_index=True)
y_total = pd.concat([y_train, y_valid], ignore_index=True).squeeze()

print(f"Tamanho total após junção: {X_total.shape[0]} amostras e {X_total.shape[1]} features")


X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(
    X_total, y_total, test_size=0.2, random_state=42, stratify=y_total
)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_final)
X_test_scaled = scaler.transform(X_test_final)


modelo = RandomForestClassifier(n_estimators=300, random_state=42)
modelo.fit(X_train_scaled, y_train_final)


# Predições normais (hard)
pred_hard = modelo.predict(X_test_scaled)

# Predições de probabilidade (soft)

prob = modelo.predict_proba(X_test_scaled)

# Simulação da votação suave
pred_soft = modelo.classes_[prob.argmax(axis=1)]


pesos = np.linspace(0.5, 1.0, prob.shape[1])
pred_weighted = modelo.classes_[np.argmax(prob * pesos, axis=1)]


def avaliar(y_true, y_pred):
    return {
        "Acurácia": accuracy_score(y_true, y_pred),
        "Precisão": precision_score(y_true, y_pred, average='weighted', zero_division=0),
        "Recall": recall_score(y_true, y_pred, average='weighted', zero_division=0),
        "F1-Score": f1_score(y_true, y_pred, average='weighted', zero_division=0),
        "Kappa": cohen_kappa_score(y_true, y_pred),
    }


resultados = {
    "Votação Majoritária": avaliar(y_test_final, pred_hard),
    "Votação Suave": avaliar(y_test_final, pred_soft),
    "Votação Ponderada": avaliar(y_test_final, pred_weighted),
}

df_resultados = pd.DataFrame(resultados).T.round(4)
print("\n============================================")
print("RESULTADOS - CENÁRIO NORMALIZADO (80/20)")
print("============================================")
print(df_resultados)


Tamanho total após junção: 2600 amostras e 500 features

RESULTADOS - CENÁRIO NORMALIZADO (80/20)
                     Acurácia  Precisão  Recall  F1-Score   Kappa
Votação Majoritária    0.7077    0.7133  0.7077    0.7058  0.4154
Votação Suave          0.7077    0.7133  0.7077    0.7058  0.4154
Votação Ponderada      0.5346    0.7340  0.5346    0.4086  0.0692


# Resultados e Discussão

Graficos 